In [204]:
import pandas as pd
from siuba import *
import numpy as np
from calitp import *

In [205]:
dla = pd.read_csv('dla_placenames.csv', index_col=[0])

In [206]:
dla = dla>>select(_.ct_code, _.dla_name, _.data_origin)

In [207]:
## importing previous NTD/ITP merge from 5310 analysis

In [288]:
m1 = pd.read_csv("ntd_itp_merge.csv")
m1.ntd_id_x.fillna(m1["ntd_id_y"], inplace=True)
m1 = m1>>select(_.ntd_id_x,
                _.agency_name_ntd,
               _.name_airtable,
               _.doing_business_as,
                _.mobility_services_operated,
                _.itp_id, 
                _._merge
)
m1['_merge'].replace({"right_only":"In Airtable", "left_only":"In NTD", "both":"In NTD, Airtable"}, inplace=True)
m1 = m1.rename(columns={'_merge':'data_origin', 'ntd_id_x':'ntd_id'})

In [289]:
m1.sample()

,ntd_id,agency_name_ntd,name_airtable,doing_business_as,mobility_services_operated,itp_id,data_origin
320,NaN,NaN,City and County Association of Governments of ...,NaN,NaN,NaN,In Airtable


In [290]:
#should come up blank
m1>>count(_.agency_name_ntd)>>arrange(-_.n)>>filter(_.n>1)

,agency_name_ntd,n


In [291]:
m1.sample()

,ntd_id,agency_name_ntd,name_airtable,doing_business_as,mobility_services_operated,itp_id,data_origin
157,90030,North County Transit District,North County Transit District,NaN,"NCTD Coaster,NCTD Sprinter,NCTD BREEZE,NCTD FLEX",226.0,"In NTD, Airtable"


In [292]:
m1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ntd_id                      242 non-null    object 
 1   agency_name_ntd             238 non-null    object 
 2   name_airtable               825 non-null    object 
 3   doing_business_as           86 non-null     object 
 4   mobility_services_operated  442 non-null    object 
 5   itp_id                      372 non-null    float64
 6   data_origin                 853 non-null    object 
dtypes: float64(1), object(6)
memory usage: 46.8+ KB


In [293]:
# condensing agency name column. using NTD first, then filling in blanks with Airtable

In [294]:
m1.agency_name_ntd.fillna(m1['name_airtable'], inplace=True)


In [295]:
m1 = m1.rename(columns={'agency_name_ntd':'name'}).drop(columns={'name_airtable'})

## Merging with DLA Locode

##### first on `name`

In [372]:
cw1 = (pd.merge(dla, m1, left_on='dla_name', right_on='name', how='outer', indicator='merge_status'))

In [373]:
cw1.merge_status.value_counts()

left_only     936
right_only    747
both          106
Name: merge_status, dtype: int64

In [374]:
cw1

,ct_code,dla_name,data_origin_x,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,data_origin_y,merge_status
0,6302.0,Humboldt Bay Harbor Recreation & Conservation ...,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,6330.0,Willow Creek Community Services District,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,5036.0,Trinidad,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,5049.0,Ukiah,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,5082.0,Willits,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
1784,NaN,NaN,NaN,NaN,Willits Seniors Inc,NaN,"Willits Paratransit,Willits Harrah Senior Center",NaN,In Airtable,right_only
1785,NaN,NaN,NaN,NaN,Work Training Center,NaN,NaN,471.0,In Airtable,right_only
1786,NaN,NaN,NaN,NaN,Worldpay,NaN,NaN,NaN,In Airtable,right_only
1787,NaN,NaN,NaN,NaN,Yellow Cab CA,NaN,"Aliso Viejo Senior Mobility Program,Garden Gro...",NaN,In Airtable,right_only


In [375]:
cw_nomatch_dla = (cw1>>filter(_.merge_status=='left_only')>>select(_.ct_code,
                                                                     _.dla_name, _.data_origin_x))


In [376]:
cw1 = (cw1>>filter(_.merge_status=='both'))
cw1.merge_status.replace({"both":"DLA match on name"}, inplace=True)

/tmp/ipykernel_1049/2744787579.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cw1.merge_status.replace({"both":"DLA match on name"}, inplace=True)


In [377]:
cw1.sample()

,ct_code,dla_name,data_origin_x,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,data_origin_y,merge_status
520,6094.0,Tulare County Association of Governments,DLA Locode,NaN,Tulare County Association of Governments,NaN,NaN,NaN,In Airtable,DLA match on name


##### second on `doing_business_as`

In [410]:
cw2 = (pd.merge(cw_nomatch_dla, m1, left_on='dla_name', right_on='doing_business_as', how='outer', indicator='merge_status'))

In [411]:
cw2.merge_status.value_counts()


left_only     932
right_only    849
both            4
Name: merge_status, dtype: int64

In [412]:
add1 = (cw2>>filter(_.merge_status=='both'))
add1['merge_status']= add1['merge_status'].replace({"both":"DLA match on doing business as"})

/tmp/ipykernel_1049/1161878024.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add1['merge_status']= add1['merge_status'].replace({"both":"DLA match on doing business as"})


In [413]:
add1

,ct_code,dla_name,data_origin_x,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,data_origin,merge_status
237,6039.0,Petaluma Transit,DLA Locode,90213,City of Petaluma,Petaluma Transit,"Petaluma Transit,Petaluma Paratransit",247.0,"In NTD, Airtable",DLA match on doing business as
289,6331.0,Sonoma County Transit,DLA Locode,90089,County of Sonoma,Sonoma County Transit,"Sonoma County Transit,Healdsburg Shuttle,Sonom...",314.0,"In NTD, Airtable",DLA match on doing business as
291,6341.0,Tri Delta Transit,DLA Locode,90162,The Eastern Contra Costa Transit Authority,Tri Delta Transit,NaN,336.0,"In NTD, Airtable",DLA match on doing business as
490,6008.0,Fresno Area Express,DLA Locode,90027,City of Fresno,Fresno Area Express,"Fresno Area Express,FAX Handy Ride",116.0,"In NTD, Airtable",DLA match on doing business as


In [414]:
cw_nomatch_dla_2 = (cw2>>filter(_.merge_status=='left_only')>>select(_.ct_code,
                                                                     _.dla_name, _.data_origin_x))

##### third on `mobility_services_operated`

In [453]:
cw3 = (pd.merge(cw_nomatch_dla_2, m1, left_on='dla_name', right_on='mobility_services_operated', how='outer', indicator='merge_status'))


In [454]:
cw3.merge_status.value_counts()


left_only     931
right_only    852
both            1
Name: merge_status, dtype: int64

In [455]:
add2 = (cw3>>filter(_.merge_status=='both'))

In [456]:
add2['merge_status']= add2['merge_status'].replace({"both":"DLA match on mobility services operated"})

/tmp/ipykernel_1049/3105742984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add2['merge_status']= add2['merge_status'].replace({"both":"DLA match on mobility services operated"})


In [457]:
add2

,ct_code,dla_name,data_origin_x,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,data_origin,merge_status
301,6295.0,Capitol Corridor,DLA Locode,NaN,Capitol Corridor Joint Powers Authority,NaN,Capitol Corridor,56.0,In Airtable,DLA match on mobility services operated


In [458]:
nomatch = (cw3>>filter(_.merge_status!='both'))

In [459]:
nomatch['merge_status']= nomatch['merge_status'].replace({"left_only":"No Match", "right_only":"No Match"})

/tmp/ipykernel_1049/1410011296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomatch['merge_status']= nomatch['merge_status'].replace({"left_only":"No Match", "right_only":"No Match"})


In [460]:
nomatch

,ct_code,dla_name,data_origin_x,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,data_origin,merge_status
0,6302.0,Humboldt Bay Harbor Recreation & Conservation ...,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,No Match
1,6330.0,Willow Creek Community Services District,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,No Match
2,5036.0,Trinidad,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,No Match
3,5049.0,Ukiah,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,No Match
4,5082.0,Willits,DLA Locode,NaN,NaN,NaN,NaN,NaN,NaN,No Match
...,...,...,...,...,...,...,...,...,...,...
1779,NaN,NaN,NaN,NaN,Vivalon Inc,NaN,Golden Gate Transit Paratransit,NaN,In Airtable,No Match
1780,NaN,NaN,NaN,NaN,Volunteer Transportation Center Inc,NaN,Volunteers on the Go,NaN,In Airtable,No Match
1781,NaN,NaN,NaN,NaN,West Berkeley Transportation Management Agency,NaN,West Berkeley Shuttle,365.0,In Airtable,No Match
1782,NaN,NaN,NaN,NaN,Willits Seniors Inc,NaN,"Willits Paratransit,Willits Harrah Senior Center",NaN,In Airtable,No Match


### Concating all matches

In [461]:
cw = pd.concat([cw1, add1], ignore_index=True, sort=False)
cw_match = pd.concat([cw, add2], ignore_index=True, sort=False)

In [462]:
cw_match.sample(5)

,ct_code,dla_name,data_origin_x,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,data_origin_y,merge_status,data_origin
62,6094.0,Tulare County Association of Governments,DLA Locode,NaN,Tulare County Association of Governments,NaN,NaN,NaN,In Airtable,DLA match on name,NaN
25,5924.0,Sacramento County,DLA Locode,NaN,Sacramento County,NaN,NaN,81.0,In Airtable,DLA match on name,NaN
87,5926.0,Amador County,DLA Locode,NaN,Amador County,NaN,NaN,NaN,In Airtable,DLA match on name,NaN
80,6391.0,Palo Verde Valley Transit Agency,DLA Locode,9R02-99454,Palo Verde Valley Transit Agency,Desert Roadrunner,Desert Roadrunner,238.0,"In NTD, Airtable",DLA match on name,NaN
49,6149.0,Santa Cruz County Regional Transportation Comm...,DLA Locode,NaN,Santa Cruz County Regional Transportation Comm...,NaN,NaN,NaN,In Airtable,DLA match on name,NaN


In [463]:
cw_match.data_origin_y.fillna(cw_match['data_origin'], inplace=True)

In [464]:
cw_match = cw_match.drop(columns='data_origin')

In [465]:
cw_match.data_origin_y.value_counts()

In Airtable         57
In NTD, Airtable    53
In NTD               1
Name: data_origin_y, dtype: int64

In [466]:
cw_match['data_origin'] = cw_match['data_origin_y'].astype(str) + ' and DLA' 

In [467]:
cw_match = cw_match.drop(columns={'data_origin_x','data_origin_y'})

In [468]:
cw_match.sample(5)

,ct_code,dla_name,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,merge_status,data_origin
44,6090.0,Santa Barbara County Association of Governments,90303,Santa Barbara County Association of Governments,NaN,Clean Air Express,379.0,DLA match on name,"In NTD, Airtable and DLA"
66,6194.0,Fresno County Rural Transit Agency,9R02-91007,Fresno County Rural Transit Agency,NaN,Sanger Express,117.0,DLA match on name,"In NTD, Airtable and DLA"
18,6225.0,El Dorado County Transit Authority,90229,El Dorado County Transit Authority,NaN,"El Dorado Transit,Sacramento/South Lake Tahoe ...",101.0,DLA match on name,"In NTD, Airtable and DLA"
95,5938.0,Stanislaus County,NaN,Stanislaus County,NaN,NaN,NaN,DLA match on name,In Airtable and DLA
102,5957.0,San Diego County,NaN,San Diego County,NaN,NaN,NaN,DLA match on name,In Airtable and DLA


In [487]:
#cw_match.to_csv('dla_ntd_itp_matches.csv')

In [469]:
#concating to have all records
cw_all = pd.concat([cw_match, nomatch], ignore_index=True, sort=False)

In [470]:
cw_all.sample(5)

,ct_code,dla_name,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,merge_status,data_origin,data_origin_x
1537,NaN,NaN,90087,City of Santa Maria,Santa Maria Area Transit,"Santa Maria Area Transit,Santa Maria Paratrans...",298.0,No Match,"In NTD, Airtable",NaN
711,5200.0,Burbank,NaN,NaN,NaN,NaN,NaN,No Match,NaN,DLA Locode
126,5145.0,Fortuna,NaN,NaN,NaN,NaN,NaN,No Match,NaN,DLA Locode
1228,NaN,NaN,NaN,City of San Carlos,NaN,NaN,NaN,No Match,In Airtable,NaN
19,6478.0,Tahoe Transportation District,NaN,Tahoe Transportation District,NaN,"Tahoe Truckee Area Regional Transportation,Tah...",331.0,DLA match on name,In Airtable and DLA,NaN


In [471]:
cw_all.data_origin.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1894 entries, 0 to 1893
Series name: data_origin
Non-Null Count  Dtype 
--------------  ----- 
963 non-null    object
dtypes: object(1)
memory usage: 14.9+ KB


In [472]:
cw_all.data_origin.fillna(cw_all['data_origin_x'], inplace=True)
cw_all.name.fillna(cw_all['dla_name'], inplace=True)
cw_all = cw_all.drop(columns={'data_origin_x', 'dla_name'})


In [473]:
cw_all.data_origin.value_counts()

DLA Locode                  931
In Airtable                 615
In NTD, Airtable            209
In Airtable and DLA          57
In NTD, Airtable and DLA     53
In NTD                       28
In NTD and DLA                1
Name: data_origin, dtype: int64

In [474]:
cw_all>>filter(_.data_origin.isnull())

,ct_code,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,merge_status,data_origin


In [475]:
cw_all.sample(5)

,ct_code,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,merge_status,data_origin
866,5230.0,NaN,Gustine,NaN,NaN,NaN,No Match,DLA Locode
1186,NaN,NaN,City of Jackson,NaN,NaN,NaN,No Match,In Airtable
164,5906.0,NaN,Shasta County,NaN,NaN,NaN,No Match,DLA Locode
1507,NaN,90175,City of Lodi,GrapeLine,"Grapeline,Vineline and Dial-A-Ride",168.0,No Match,"In NTD, Airtable"
896,5081.0,NaN,Escondido,NaN,NaN,NaN,No Match,DLA Locode


In [476]:
cw_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ct_code                     1042 non-null   float64
 1   ntd_id                      298 non-null    object 
 2   name                        1894 non-null   object 
 3   doing_business_as           99 non-null     object 
 4   mobility_services_operated  497 non-null    object 
 5   itp_id                      433 non-null    float64
 6   merge_status                1894 non-null   object 
 7   data_origin                 1894 non-null   object 
dtypes: float64(2), object(6)
memory usage: 118.5+ KB


In [477]:
cw_all>>filter(_.mobility_services_operated.notnull())

,ct_code,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,merge_status,data_origin
0,6241.0,9R02-91047,Mendocino Transit Authority,NaN,"Mendocino Transit Authority,Ukiah Dial-A-Ride,...",198.0,DLA match on name,"In NTD, Airtable and DLA"
3,6428.0,9R02-91097,Redwood Coast Transit Authority,NaN,"Redwood Coast Transit,Redwood Coast Dial-A-Ride",261.0,DLA match on name,"In NTD, Airtable and DLA"
4,6436.0,9R02-91053,Lake Transit Authority,NaN,"Lake Transit,Clearlake/Lower Lake Dial-A-Ride,...",159.0,DLA match on name,"In NTD, Airtable and DLA"
5,6162.0,9R02-91036,Humboldt Transit Authority,NaN,"Humboldt Transit Authority Dial-A-Ride,Redwood...",135.0,DLA match on name,"In NTD, Airtable and DLA"
6,5908.0,9R02-91089,Tehama County,NaN,"Tehama Rural Area eXpress,ParaTRAX",334.0,DLA match on name,"In NTD, Airtable and DLA"
...,...,...,...,...,...,...,...,...
1889,NaN,NaN,Vivalon Inc,NaN,Golden Gate Transit Paratransit,NaN,No Match,In Airtable
1890,NaN,NaN,Volunteer Transportation Center Inc,NaN,Volunteers on the Go,NaN,No Match,In Airtable
1891,NaN,NaN,West Berkeley Transportation Management Agency,NaN,West Berkeley Shuttle,365.0,No Match,In Airtable
1892,NaN,NaN,Willits Seniors Inc,NaN,"Willits Paratransit,Willits Harrah Senior Center",NaN,No Match,In Airtable


In [478]:
cw_all.merge_status.value_counts()

No Match                                   1783
DLA match on name                           106
DLA match on doing business as                4
DLA match on mobility services operated       1
Name: merge_status, dtype: int64

In [480]:
cw_all>>filter(_.merge_status=='DLA match on mobility services operated')

,ct_code,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,merge_status,data_origin
110,6295.0,NaN,Capitol Corridor Joint Powers Authority,NaN,Capitol Corridor,56.0,DLA match on mobility services operated,In Airtable and DLA


In [481]:
cw_all>>filter(_.doing_business_as.notnull())>>filter(_.merge_status=='DLA match on doing business as')

,ct_code,ntd_id,name,doing_business_as,mobility_services_operated,itp_id,merge_status,data_origin
106,6039.0,90213,City of Petaluma,Petaluma Transit,"Petaluma Transit,Petaluma Paratransit",247.0,DLA match on doing business as,"In NTD, Airtable and DLA"
107,6331.0,90089,County of Sonoma,Sonoma County Transit,"Sonoma County Transit,Healdsburg Shuttle,Sonom...",314.0,DLA match on doing business as,"In NTD, Airtable and DLA"
108,6341.0,90162,The Eastern Contra Costa Transit Authority,Tri Delta Transit,NaN,336.0,DLA match on doing business as,"In NTD, Airtable and DLA"
109,6008.0,90027,City of Fresno,Fresno Area Express,"Fresno Area Express,FAX Handy Ride",116.0,DLA match on doing business as,"In NTD, Airtable and DLA"


In [482]:
cw_all = cw_all[['ct_code', 'ntd_id', 'itp_id', 'name', 'merge_status', 'data_origin']]

In [483]:
cw_all

,ct_code,ntd_id,itp_id,name,merge_status,data_origin
0,6241.0,9R02-91047,198.0,Mendocino Transit Authority,DLA match on name,"In NTD, Airtable and DLA"
1,6140.0,NaN,NaN,Mendocino Council of Governments,DLA match on name,In Airtable and DLA
2,6133.0,NaN,NaN,Humboldt County Association of Governments,DLA match on name,In Airtable and DLA
3,6428.0,9R02-91097,261.0,Redwood Coast Transit Authority,DLA match on name,"In NTD, Airtable and DLA"
4,6436.0,9R02-91053,159.0,Lake Transit Authority,DLA match on name,"In NTD, Airtable and DLA"
...,...,...,...,...,...,...
1889,NaN,NaN,NaN,Vivalon Inc,No Match,In Airtable
1890,NaN,NaN,NaN,Volunteer Transportation Center Inc,No Match,In Airtable
1891,NaN,NaN,365.0,West Berkeley Transportation Management Agency,No Match,In Airtable
1892,NaN,NaN,NaN,Willits Seniors Inc,No Match,In Airtable


In [485]:
#cw_all.to_csv('dla_ntd_itp_crosswalk.csv')